
 # Connecting Networks
* **scikit-rf** supports the connection of arbitrary ports of N-port networks. 

<center><img src="figures/circuit_general.svg" width="400"/></center>

* Note that port impedances are taken into account: if two connected ports have different port impedances, an appropriate impedance mismatch is inserted.

## Cascading Networks
A common problem is to connect two Networks one to the other, also known as cascading Networks, which creates a new Network. The figure below illustrates sile simple situations, where the port numbers are identified in gray:

<img src="figures/networks_connecting_2_2ports.svg" width="600">

or,

<img src="figures/networks_connecting_2port_1port.svg" width="600">

Let's illustrate this by connecting a transmission line (2-port Network) to a short-circuit (1-port Network) to create a delay short (1-port Network):

<img src="figures/networks_delay_short.svg" width="600">

Cascading Networks being a frequent operation, it can done conveniently through the `**` operator or with the `cascade` function: 

In [ ]:
import skrf as rf
line = rf.data.wr2p2_line  # 2-port
short = rf.data.wr2p2_short  # 1-port
delayshort = line ** short  # --> 1-port Network
print(delayshort)

or, equivalently using the `cascade()` function:

In [ ]:
delayshort2 = rf.cascade(line, short)
print(delayshort2 == delayshort)  # the result is the same

It is of course possible to connect two 2-port Networks together: 

<center><img src="figures/cascade_2port_2port.png", height="400"/></center>

In [ ]:
double_line = line ** line  # or cascade(line, line)
print(double_line)

In [ ]:
line.plot_s_deg_unwrap(m=1, n=0)
double_line.plot_s_deg_unwrap(m=1, n=0)

## Connecting Networks with `connect()`
The `connect()` function requires the Networks and the port numbers to connect together. In our example, the port 1 of the line is connected to the port 0 of the short: 
<img src="figures/networks_delay_short.svg" width="600">


In [ ]:
delayshort3 = rf.connect(line, 1, short, 0)
print(delayshort3 == delayshort)

One often needs to cascade a chain Networks together:

<img src="figures/networks_connecting_N_2ports.svg" width="700">
or, 
<img src="figures/networks_connecting_N_2ports_1port.svg" width="700">


which can be realized using chained `**` or the convenient function `cascade_list`:

In [ ]:
line1 = rf.data.wr2p2_line  # 2-port
line2 = rf.data.wr2p2_line  # 2-port
line3 = rf.data.wr2p2_line  # 2-port
line4 = rf.data.wr2p2_line  # 2-port
short = rf.data.wr2p2_short  # 1-port
chain1 = line1 ** line2 ** line3 ** line4 ** short
chain2 = rf.cascade_list([line1, line2, line3, line4, short])
print(chain1 == chain2) 

## Cascading 2N-port Networks
The cascading operator `**` also works for to 2N-port Networks, width the following port scheme: 

<img src="figures/networks_connecting_2_2Nports.svg" width="1000">

It also works for multiple 2N-port Network. For example, assuming you want to cascade three 4-port Network `ntw1`, `ntw2` and `ntw3`, you can use:
```
resulting_ntw = ntw1 ** ntw2 ** ntw3
``` 

## Cascading Multi-port Networks
To make specific connections between multi-port Networks, two solutions are available, which mostly depends of the complexity of the circuit one wants to build:

* For reduced number of connection(s): the `connect()` function

* For advanced connections between many arbitrary N-port Networks, the `Circuit` object is more relevant since it allows defining explicitly the connections between ports and Networks. (cf. Next Section)

### Example: connecting a Tee
As an example, terminating one of the port of an a 3-port Network, such as an ideal 3-way splitter:

<img src="figures/networks_connecting_3port_1port.svg" width="1000">

can be done like:

To connect port `1` of the tee, to port `0` of the delay short,


In [ ]:
tee = rf.data.tee
terminated_tee = rf.connect(tee, 1, delayshort, 0)
terminated_tee 

In the previous example, the port #2 of the 3-port Network `tee` becomes the port #1 of the resulting 2-port Network. 

## Multiple Connections of Multi-port Networks with  `Connect()`
Keeping track of the port numbering when using multiple time the `connect` function can be tedious... 

...this is the reason why the Circuit object can be simpler to use.

Let's illustrate this with the following example: connecting the port #1 of a tee-junction (3-port) to the port #0 of a transmission line (2-port):

<img src="figures/networks_connecting_3port_2port.svg" width="600">

To keep track of the port scheme after the connection operation, let's change the port characteristic impedances (in red in the figure above):

In [ ]:
tee.z0 = [1, 2, 3]
line.z0 = [10, 20]
# the resulting network is:
rf.connect(tee, 1, line, 0)

# Advanced Circuits with `Circuit()`
<img src="figures/circuit_general.svg" align="right" width="500"/>

**scikit-rf** allows creating circuits of arbitrary topology, consisting of an arbitrary number of N-ports Networks connected together.

The figure illustrates a network with 2 ports, `Network` elements $N_i$ and intersections.

<img src="figures/circuit_general.svg" align="right" width="500"/>

* Like in an electronic circuit simulator, the circuit must have one (or more) `Port` connected to the circuit. 
* The `Circuit` object allows one retrieving the M-ports `Network` (and thus its network parameters: $S$, $Z$, etc.), where M is the number of ports defined.
* Moreover, the `Circuit` object also allows calculating the scattering matrix $S$ of the entire circuit, that is the "internal" scattering matrices for the various intersections in the circuit. 

## The Connection List 
The connection list ("netlist") is defined as a List of List of interconnected Tuples `(network, port_number)`:

```
connexions = [
    [(network1, network1_port_nb), (network2, network2_port_nb), (network2, network2_port_nb), ...],
    ...
]
```

For example, the connexion list to construct the circuit above could be: 
<img src="figures/circuit_general.svg" align="right" width="500"/>
```
connexions = [
    [(port1, 0), (network1, 0), (network4, 0)],
    [(network1, 1), (network2, 0), (network5, 0)],
    [(network1, 2), (network3, 0)],
    [(network2, 1), (network3, 1)],
    [(network2, 2), (port2, 0)],
    [(network5, 1), (ground1, 0)],
    [(network4, 1), (open1, 0)]
]
```

* We have assumed that `port1`, `port2`, `ground1`, `open1` and all the `network1` to `network5` are scikit-rf Networks objects with same `Frequency`.
* Networks can have different (real) characteristic impedances: mismatches are taken into account.

* Convenience methods are provided to create ports and grounded connexions:

 * `Circuit.Port`
 * `Circuit.Ground`
 * `Circuit.SeriesImpedance`
 * `Circuit.ShuntAdmittance`
 * `Circuit.Open`

* Once the connexion list is defined, the `Circuit` is created with:

```
resulting_circuit = rf.Circuit(connexions)
```
where `resulting_circuit` is a `Circuit` object. 

* The resulting 2-ports `Network` is obtained with the `Circuit.network` parameter:

```
resulting_network = resulting_circuit.network
```

## Example: Loaded transmission line
Assume that a $50\Omega$ lossless transmission line is loaded with a $Z_L=75\Omega$ impedance. 

<img src="figures/circuit_loaded_transmission_line1.svg" width="800"/>

If the transmission line electric length is $\theta=0$, then one would thus expect the reflection coefficient to be:

$$
\rho = s = \frac{Z_L - Z_0}{Z_L + Z_0} = 0.2
$$

In [ ]:
Z_0 = 50
Z_L = 75
theta = 0

# the necessary Frequency description
freq = rf.Frequency(start=1, stop=2, unit='GHz', npoints=3)

In [ ]:
# The combination of a transmission line + a load can be created
# using the convenience delay_load method
# important: all the Network must have a unique "name" parameter
tline_media = rf.DefinedGammaZ0(freq, z0=Z_0)
delay_load = tline_media.delay_load(rf.zl_2_Gamma0(Z_0, Z_L), theta, 
                                    unit='deg', name='delay_load')

In [ ]:
# The input port of the circuit is defined with the Circuit.Port method
port1 = rf.Circuit.Port(freq, 'port1', z0=Z_0)

# connection list
cnx = [
    [(port1, 0), (delay_load, 0)]
]

In [ ]:
# Building the circuit object
cir = rf.Circuit(cnx)

# Getting the resulting Network from the `network` parameter:
ntw = cir.network
print(ntw)

In [ ]:
# as expected the reflection coefficient is:
print(ntw.s[0])

It is also possible to build the above circuit using a series impedance Network, then shorted:

<img src="figures/circuit_loaded_transmission_line2.svg" width="80%"/>


In [ ]:
port1 = rf.Circuit.Port(freq, 'port1', z0=Z_0)
# piece of transmission line and series impedance
trans_line = tline_media.line(theta, unit='deg', name='trans_line')
load = tline_media.resistor(Z_L, name='delay_load')
# ground network (short)
ground = rf.Circuit.Ground(freq, name='ground')

# connexion list
cnx = [
    [(port1, 0), (trans_line, 0)],
    [(trans_line, 1), (load, 0)],
    [(load, 1), (ground, 0)]
]
# building the circuit
cir = rf.Circuit(cnx)
# the result if the same : 
print(cir.network.s[0])

## Example: LC Filter
Here we model a low-pass LC filter, with example values taken from [rf-tools.com](https://rf-tools.com/lc-filter/) :

<img src="figures/circuit_filter1.svg" width="50%">

In [ ]:
freq = rf.Frequency(start=0.1, stop=10, unit='GHz', npoints=1001)
tl_media = rf.DefinedGammaZ0(freq, z0=50, gamma=1j*freq.w/rf.c)
C1 = tl_media.capacitor(3.222e-12, name='C1')
C2 = tl_media.capacitor(82.25e-15, name='C2')
C3 = tl_media.capacitor(3.222e-12, name='C3')
L2 = tl_media.inductor(8.893e-9, name='L2')
RL = tl_media.resistor(50, name='RL')
gnd = rf.Circuit.Ground(freq, name='gnd')
port1 = rf.Circuit.Port(freq, name='port1', z0=50) 
port2 = rf.Circuit.Port(freq, name='port2', z0=50)

cnx = [
    [(port1, 0), (C1, 0), (L2, 0), (C2, 0)],
    [(L2, 1), (C2, 1), (C3, 0), (port2, 0)],
    [(gnd, 0), (C1, 1), (C3, 1)],
]
cir = rf.Circuit(cnx)
ntw = cir.network

In [ ]:
ntw.plot_s_db(m=0, n=0, lw=2, logx=True)
ntw.plot_s_db(m=1, n=0, lw=2, logx=True)

## Displaying Circuit's graph
When building a `Circuit` made of several networks, it can be useful to represent the connexion graphically. This is possible using the `Circuit.plot_graph()`method. 

* Ports are indicated by triangles, 
* Network with squares
* interconnections with circles. 
* It is possible to display the network names as well as their associated ports (and characteristic impedances): 

In [ ]:
cir.plot_graph(network_labels=True, network_fontsize=15, port_labels=True, port_fontsize=15,
              edge_labels=True, edge_fontsize=10)